__Document Q&A - Retrieval with citations__



In [1]:
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


In [2]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
# Load environment variables
load_dotenv()

# Initialize Groq chat
groq_chat = ChatGroq(
    temperature=0.7,
    model_name="llama-3.1-8b-instant",  # Groq's model
    api_key=os.getenv("GROQ_API_KEY")
)

In [3]:

# Simulate document processing
documents = [
    "Our company policy states that employees get 20 vacation days per year.",
    "Healthcare benefits include medical, dental, and vision coverage starting after 90 days.",
    "The work from home policy allows up to 3 days remote work per week for eligible positions.",
    "Performance reviews are conducted bi-annually in June and December.",
    "All employees must complete security training within their first 30 days."
]

In [4]:

# Create embeddings and vector store
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs = text_splitter.create_documents(documents)
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(k=3)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7016\3408348619.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7016\3408348619.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Lenovo\anaconda3\envs\agentvenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:
# Create Q&A chain with citation
qa_prompt = ChatPromptTemplate.from_template("""
Answer the question based only on the following context. 
If you don't know the answer, say you don't know. 
Always cite which document number you got the information from.

Context:
{context}

Question: {question}

Answer with citation:
""")

# Function to format documents with their indices
def format_docs(docs):
    return "\n\n".join([f"Document {i+1}: {doc.page_content}" for i, doc in enumerate(docs)])

In [ ]:
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | qa_prompt
    | groq_chat
    | StrOutputParser()
)

# Test the chain
questions = [
    "How many vacation days do employees get?",
    "When do healthcare benefits start?",
    "What is the work from home policy?"
]



In [ ]:
# for question in questions:
#     print(f"Q: {question}")
#     answer = qa_chain.invoke(question)
#     print(f"A: {answer}\n")
answer = qa_chain.invoke("What is the work from home policy?")
